In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'philstar_com'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_rss_page = 0
publish_source = 'philstar.com'
country = 'Philippines'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

rss_pages = c.download_page('https://www.philstar.com/rss')
rss_pages = c.scrap('<div\s*id="rss-container">(.*?)<div\s*id="nav_footer">', rss_pages)

for _ in rss_pages.split('<li>')[1:]:
    
    rss_url = c.scrap('href="(.*?)"', _)
    if 'http' not in rss_url:
        rss_url = 'https://www.philstar.com/' +rss_url

    logger.info(f'Fetching rss_url  {rss_url}\n')
    rss = c.download_page(rss_url)
    
    if rss.startswith('Unable to fetch'):
        logger.info(rss) # writes error message with error code
        unable_to_fetch_rss_page += 1
        continue    

    for i in rss.split('<item>')[1:]:

        # source_link
        source_link = c.scrap("<link>(.*?)<", i)
        
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
        
        # source_headline
        source_headline = c.scrap('<title>(.*?)<', i)
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
        
        # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap("<pubDate>(.*?)\+", i)         
            date_time_obj = datetime.strptime(date_time_str, '%a, %d %b %Y %H:%M:%S')
            # Philippines is 2 hours and 30 minutes ahead of India
            ist_date_time = date_time_obj - timedelta(hours = 2,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        if pub_date != created_on:
            break    

        logger.info(f'Fetching {source_link}\n')

        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
            logger.info(page) # writes error message with error code
            unable_to_fetch_url += 1
            continue    

        # source_content          
        source_content = c.scrap('id="sports_article_writeup">(.*?)<div\s*class="facebook-philstar-embed".*?>', page)
        if not source_content:
            source_content = c.scrap('<p>(.*?)<strong>', page) 
        if not source_content:
            source_content = c.scrap('<meta\s*name="description" content="(.*?)<div\s*id="respond">', page) 
        source_content = re.sub ('&#39;|&rsquo;|&lsquo;', "'", source_content)
        source_content = re.sub ('&ldquo;|&rdquo;|&quot;', '"', source_content)
        source_content = re.sub ('&mdash;', '-', source_content)
        source_content = c.strip_html(source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist = c.scrap("<author>(.*?)<", i)
        if not journalist: journalist = 'NA'
       
        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_page}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://www.philstar.com/rss/headlines
https://www.philstar.com/rss/opinion
https://www.philstar.com/rss/nation
https://www.philstar.com/rss/world
https://www.philstar.com/rss/business
https://www.philstar.com/rss/sports
https://www.philstar.com/rss/entertainment
https://www.philstar.com/rss/lifestyle
https://www.philstar.com/rss/pilipino-star-ngayon/bansa
https://www.philstar.com/rss/pilipino-star-ngayon/probinsiya
https://www.philstar.com/rss/pilipino-star-ngayon/metro
https://www.philstar.com/rss/pilipino-star-ngayon/opinyon
https://www.philstar.com/rss/pilipino-star-ngayon/palaro
https://www.philstar.com/rss/pilipino-star-ngayon/showbiz
https://www.philstar.com/rss/the-freeman
https://www.philstar.com/rss/pang-masa
https://www.philstar.com/rss/banat
https://interaksyon.philstar.com/trends-spotlights/feed/
https://interaksyon.philstar.com/politics-issues/feed/
https://interaksyon.philstar.com/celebrities/feed/
https://interaksyon.philstar.com/hobbies-interests/feed/
https://interaks